In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
breweries= pd.read_csv('breweries.csv')

In [4]:
url_list=[ x for x in breweries['Untappd']]
brewery_list=[ x for x in breweries['Brewery']]

In [14]:
#url_list

In [15]:
#breweries['Untappd']

In [16]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [17]:
brewery_id = []
x=0
for brewery in breweries['Brewery']:
    brewery_id += 5 * [x]
    x +=1
#brewery_id

In [37]:
beer_name=[]
beer_type=[]
beer_abv=[]
beer_ibu=[]
beer_rating=[]
beer_number_rating=[]

for url in url_list:
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    beer = soup.find('div', class_='beer-container')
    beer_name_search = beer.find_all('p', class_='name')[:5]
    beer_type_search = beer.find_all('p', class_='style')[:5]
    beer_abv_search = beer.find_all('div', class_='details-item abv')[:5]
    beer_ibu_search = beer.find_all('div', class_='details-item ibu')[:5]
    beer_rating_search = beer.find_all('span', class_='num')[:5]
    beer_popular = beer.find_all('div', class_='details-item raters')[:5]
    for paragraph in beer_name_search:
        if paragraph.a:
            beer_name.append(paragraph.a.text) 
    for paragraph in beer_type_search:
        if paragraph.text:
            beer_type.append(paragraph.text)
    for paragraph in beer_abv_search:
        if paragraph.text:
            beer_abv.append(paragraph.text)
    for paragraph in beer_ibu_search:
        if paragraph.text:
            beer_ibu.append(paragraph.text)
    for paragraph in beer_rating_search:
        if paragraph.text:
            beer_rating.append(paragraph.text)
    for paragraph in beer_popular:
        if paragraph.text:
            beer_number_rating.append(paragraph.text)

In [38]:
print(len(beer_name))
print(len(beer_type))
print(len(beer_abv))
print(len(beer_ibu))
print(len(beer_rating))
print(len(beer_number_rating))

280
280
280
280
280
280


In [34]:
search_url = breweries['Untappd'][1]
browser.visit(search_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
beer = soup.find('div', class_='beer-container')

In [20]:
beer = soup.find('div', class_='beer-container')
beer_name_search = beer.find_all('p', class_='name')[:5]
beer_type_search = beer.find_all('p', class_='style')[:5]
beer_abv_search = beer.find_all('div', class_='details-item abv')[:5]
beer_ibu_search = beer.find_all('div', class_='details-item ibu')[:5]
beer_rating_search = beer.find_all('span', class_='num')[:5]
beer_popular = beer.find_all('div', class_='details-item raters')[:5]

In [31]:
beer_name = []
for paragraph in beer_name_search:
        if paragraph.a:
            beer_name.append(paragraph.a.text)

In [40]:
#beer_name

In [21]:
beer_abv=[]
for paragraph in beer_abv_search:
    if paragraph.text:
        beer_abv.append(paragraph.text)

In [22]:
beer_abv

['\n5.8% ABV ', '\n9.3% ABV ', '\n7.8% ABV ', '\n5.6% ABV ', '\n10.9% ABV ']

In [23]:
beer_ibu=[]
for paragraph in beer_ibu_search:
    if paragraph.text:
        beer_ibu.append(paragraph.text)

In [24]:
beer_ibu

['\nN/A IBU ', '\nN/A IBU ', '\n36 IBU ', '\n33 IBU ', '\nN/A IBU ']

In [25]:
beer_abv=[]
for paragraph in beer_abv_search:
    if paragraph.a:
        beer_name.append(paragraph.a.text)

In [27]:
beer_name

[]

In [28]:
beer_type=[]
for paragraph in beer_type_search:
    if paragraph.text:
        beer_type.append(paragraph.text)    

In [29]:
beer_type

['Wheat Beer - Witbier',
 'Belgian Tripel',
 'Farmhouse Ale - Saison',
 'Farmhouse Ale - Saison',
 'Belgian Quadrupel']

In [30]:
beer_rating=[]
for paragraph in beer_rating_search:
    if paragraph.text:
        beer_rating.append(paragraph.text)

In [ ]:
beer_rating

In [ ]:
beer_popular

In [ ]:
beer_number_rating=[]
for paragraph in beer_popular:
    if paragraph.text:
        beer_number_rating.append(paragraph.text)

In [ ]:
beer_number_rating

In [ ]:
beer_name
beer_type
beer_rating
beer_number_rating

In [ ]:
len(beer_number_rating)

In [ ]:
#brewery_id
#name
#type
#abv
#ibu
#rating
#rating_count
